In [1]:
# Standard libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "neuronal-sequence-test"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

# User libraries
from src.lib.qt_wrapper import gui_fnames, gui_fpath
from src.lib.data_db import BehaviouralNeuronalDatabase

%load_ext autoreload
%autoreload 2

Appended root directory /media/aleksejs/DataHDD/work/codes/comp-neuro/analysis-neuro/neuronal-sequence-test


# TODO

1. Preprocessing : Apply background subtraction
2. Alignment : Ensure intervals actually match. Ask Maria for visual test of alignment
3. Apply metrics - 1 data point per trial - pool trials - maintenance vs all
    * 1D semistatic: Mean, Variance, 1D Entropy
    * ND semistatic: Avg Correlation, ND Entropy
    * 1D dynamic: Predictive Info, AR(1) efficiency
    * ND dynamic: Predictive Info, AR(1) efficiency, Orderability coeff
    
4. Test Hypotheses:
    1. Maintenance predicts Correct/Incorrect vs other phases
    2. Maintenance predicts L/R better vs other phases
    3. Validation: Prev trial C/I or L/R predicts that of next trial
    4. Predictive aspect:
        * Semi-static maintenance activity is good predictor, fine temporal details irrelevant
        * Semi-static maintenance activity is not good, fine temporal details improve prediction

In [2]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
#params['root_path_data']  = gui_fpath("Path to data files", "./")
params['root_path_data'] = '/media/aleksejs/DataHDD/work/data/maria'

In [3]:
dataDB = BehaviouralNeuronalDatabase(params)

In [12]:
dataDB.read_neuro_files()

IntProgress(value=0, description='Read Neuro Data:', max=14)

In [13]:
dataDB.read_behavior_files()

IntProgress(value=0, description='Read Neuro Data:', max=14)

Reading /media/aleksejs/DataHDD/work/data/maria/m060/Behavior_m060_20190522.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m060/Behavior_m060_20190524.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m060/Behavior_m060_20190527.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m060/Behavior_m060_20190606.mat
No trials found for Trial_LWhole_Mistake skipping
Reading /media/aleksejs/DataHDD/work/data/maria/m065/Behavior_m065_20191128.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m065/Behavior_m065_20191204.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m065/Behavior_m065_20191205.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m065/Behavior_m065_20191206.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m065/Behavior_m065_20191209.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m068/Behavior_m068_20200130.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m068/Behavior_m068_20200131.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m068/Behavior_m068_202

In [23]:
dataSets = {
    "data_L_C_mt_raw"    : dataDB.get_data_mouse_phase("m060", "Maintenance", "L", "Correct", "raw"),
    "data_L_C_mt_high"   : dataDB.get_data_mouse_phase("m060", "Maintenance", "L", "Correct", "high"),
    "data_L_C_all_raw"   : dataDB.get_data_mouse_interval("m060", 0, 17, "L", "Correct", "raw"),
    "data_L_C_all_high"  : dataDB.get_data_mouse_interval("m060", 0, 17, "L", "Correct", "high")    
}

In [25]:

for performance in ["Correct", "Mistake"]:
    for direction in ["L", "R"]:
        for dataType in ["raw", "high"]:
            dataAll = dataDB.get_data_mouse_phase("m060", None, direction, performance, dataType)
            dataMt = dataDB.get_data_mouse_phase("m060", "Maintenance", direction, performance, dataType)
            
            dataAllFlat = np.vstack(dataAll)
            dataMtFlat = np.vstack(dataMt)
            
            print(np.mean(dataMtFlat), "vs", np.mean(dataAllFlat))
    
    

IndexError: index 18 is out of bounds for axis 1 with size 18